In [ ]:
from symbol import Symbol
from portifolio import Portifolio
import pandas as pd
import numpy as np

In [136]:
symbol = Symbol(['willamagor', 'windmachin', 'hindalco'], start=1996)
port = Portifolio(symbol_list='infy', benchmark='nifty_50')
traded_dates = pd.read_hdf('constants.h5', 'traded_dates')

In [137]:
data = symbol.get_symbol_hist()

In [138]:
data = data[[
    'symbol', 'date', 'prev_close', 'open', 'high',
    'low', 'last', 'close', 'vwap',
    'trades', 'volume', 'turnover', 'pct_deliverble'
]]

In [142]:
def descriptive_statistics(data):
    data = data.set_index('date')
    descriptive_stats = pd.Series()
    descriptive_stats['from_date'] = data.index.min()
    descriptive_stats['to_date'] = data.index.max()
    descriptive_stats['count'] = len(data.index)
    stock_sprecific_traded_dates = traded_dates[descriptive_stats['from_date']:]
    total_trades = stock_sprecific_traded_dates.join(data).drop(['days_count'], axis=1)
    missing_dates = total_trades.index.difference(data.index)
    missing_count = len(missing_dates)
    descriptive_stats['missing_count'] = missing_count

    if missing_count > 0:
        non_null_close = total_trades[['prev_close', 'close']].dropna()
        if len(non_null_close[non_null_close.close.shift(1) != non_null_close.prev_close]) <= 1:
            total_trades[['symbol', 'close',]] = total_trades[['symbol', 'close',]].ffill()
            total_trades.prev_close = total_trades.prev_close.fillna(total_trades.close)
            total_trades.open = total_trades.open.fillna(total_trades.close)
            total_trades.high = total_trades.high.fillna(total_trades.close)
            total_trades.low = total_trades.low.fillna(total_trades.close)
            total_trades['last'] = total_trades['last'].fillna(total_trades.close)
            total_trades.vwap = total_trades.vwap.fillna(total_trades.close)
            total_trades.volume = total_trades.volume.fillna(0)
            total_trades.turnover = total_trades.turnover.fillna(0)
            descriptive_stats['missing_count'] = 0
        else:
            print(non_null_close[non_null_close.close.shift(1) != non_null_close.prev_close])
    return descriptive_stats

In [143]:
des = data.groupby('symbol').apply(descriptive_statistics)

            prev_close    close
date                           
1996-01-01      923.10   919.00
1996-07-08        0.00  1191.00
1996-07-22     1146.60  1138.25
1997-03-01      744.90   853.90
1998-08-25      523.45   474.15
1998-08-31      440.90   429.15
1999-01-11      568.00   567.50
1999-05-20      580.00   586.00
2006-03-10       83.55    84.50
2006-03-13      160.00   159.50
2006-07-24      157.45   155.35
2006-11-30      176.10   173.15
2007-04-10      135.95   136.40
2007-08-20      140.05   141.85
            prev_close    close
date                           
1996-01-01      923.10   919.00
1996-07-08        0.00  1191.00
1996-07-22     1146.60  1138.25
1997-03-01      744.90   853.90
1998-08-25      523.45   474.15
1998-08-31      440.90   429.15
1999-01-11      568.00   567.50
1999-05-20      580.00   586.00
2006-03-10       83.55    84.50
2006-03-13      160.00   159.50
2006-07-24      157.45   155.35
2006-11-30      176.10   173.15
2007-04-10      135.95   136.40
2007-08-

In [141]:
['1996-04-01', '1996-04-05', '1996-07-17', '1996-10-30',
'1997-03-28', '1997-07-18', '1997-10-03', '1997-10-06',
'1997-10-07', '1997-10-08', '1997-11-14', '1998-02-16',
'1998-04-08', '1998-11-25', '2000-01-01']
des

,from_date,to_date,count,missing_count
symbol,,,,
hindalco,1996-01-01 00:00:00,2017-01-30 00:00:00,5044,222
willamagor,1997-02-24 00:00:00,2017-01-30 00:00:00,3691,1286
windmachin,2011-08-17 00:00:00,2017-01-30 00:00:00,1306,0


In [28]:
data[data.date == '2013-07-12']

,symbol,date,prev_close,open,high,low,last,close,vwap,simple_returns,log_returns,daily_volatility,trades,volume,turnover,pct_deliverble
1518681,hindalco,2013-07-12,102.8,103.1,103.5,100.7,101.8,101.8,101.73,-0.009728,-0.009775,0.002394,25961.0,4027737.0,409730000.0,31.11


In [ ]:
iret = port.get_index_returns()
sret = symbol.get_symbol_data()
index = iret.index.union(sret.index)
date_range = pd.date_range(start=index[0], end=index[-1])
business_range = pd.date_range(start=index[0], end=index[-1], freq='B')

In [ ]:
date_range.difference(index)
business_range.difference(index)
index.difference(business_range)

In [ ]:
traded_dates = pd.DataFrame(np.arange(len(index))+1, index=index.copy(), columns=['days_count'])
traded_dates = traded_dates.ix[0:-1]
traded_dates.head()

In [ ]:
traded_dates.to_hdf('constants.h5', 'traded_dates')

In [144]:
idata = data[data.symbol == 'willamagor'].set_index('date')

In [146]:
idata['2001-11':'2001-12']

,symbol,prev_close,open,high,low,last,close,vwap,trades,volume,turnover,pct_deliverble
date,,,,,,,,,,,,
2001-11-12,willamagor,12.90,12.9,12.9,12.9,12.9,12.9,12.90,NaN,300.0,4000.0,NaN
2001-11-20,willamagor,11.25,12.6,12.6,12.6,12.6,12.6,12.60,NaN,100.0,1000.0,NaN
2001-12-18,willamagor,12.60,10.0,10.0,10.0,10.0,10.0,10.00,NaN,15.0,0.0,NaN
2001-12-21,willamagor,10.00,9.0,9.0,8.0,8.0,8.0,8.67,NaN,30.0,0.0,NaN
2001-12-31,willamagor,12.90,6.5,6.5,6.5,6.5,6.5,6.50,NaN,200.0,1000.0,NaN


In [ ]:
data

In [ ]:
store = pd.HDFStore('symbol_data.h5')

In [ ]:
del store['symbol_hist_data']

In [ ]:
fetc